# <div style="text-align:center; border: 2px solid #FFA500; border-radius: 25px"><span style="color:purple">Pawpularity Prediction</span></div>
In this competition, our goal is to predict the engaggement with a pet's profile based on the appearance of that profile. For example, what kind of pictures are likely to be attracted by someone. (including the pet's name, using props in the picture, using multiple pictures, using accessories etc.)

We are provide with metadata (data about data)
- Image data
- Tabular data


*We want to predict the **Pawpularity score**. We can expect pets with attractive photos to generate more interest and be adopted faster.*
 

# Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import random
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
import lightgbm as lgb

pd.set_option('display.max_colwidth',None)

loading the `csv` files:

In [ ]:
train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
sample_submission = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')

In [ ]:
train.head(2)

In [ ]:
test.head(2)

In [ ]:
sample_submission.head(2)

In [ ]:
print(train.shape, test.shape, sample_submission.shape)

# Exploratory Data Analysis

[**My EDA Kernel**](https://www.kaggle.com/esratmaria/petfinder-pawpularity-prediction-eda)

# Model Creation

In [ ]:
models = []
columns = [col for col in train.columns if col not in ['Id', 'Pawpularity']]
X = train[columns]
y = train['Pawpularity']

In [ ]:
len(X)

In [ ]:
X_test = test[columns]
X_test

In [ ]:
print(X.shape, y.shape, X_test.shape)

In [ ]:
X = np.array(X)
X_test = np.array(X_test)
y = y.to_numpy()

In [ ]:
print(X.shape, y.shape, X_test.shape)

In [ ]:
params = {'n_estimators': 2415,
 'learning_rate': 0.011742909924093024,
 'num_leaves': 737,
 'max_depth': 11,
 'feature_fraction': 0.9871498325775747,
 'lambda_l1': 21.717215991758113,
 'lambda_l2': 7.849282580359565e-05,
 'boosting_type': 'gbdt',
 'objective': 'regression',
 'metric': {'rmse'}}

In [ ]:
folds = KFold(n_splits=5, shuffle=True, random_state=2021)

for fold_n, (train_index, valid_index) in tqdm(enumerate(folds.split(X, y))):
    print(f'------------------ Fold {fold_n} ------------------')
    X_train, X_valid =  X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_valid, y_valid)
    model = lgb.train(
        params,
        lgb_train,
        num_boost_round=5000,
        valid_sets=(lgb_train, lgb_val),
        early_stopping_rounds=100,
        verbose_eval=100
    )
    models.append(model)

# Prediction

In [ ]:
prediction = []

In [ ]:
for model in models:
    prediction.append(model.predict(test[columns]))

In [ ]:
prediction

In [ ]:
mean = np.mean(prediction, axis=0)
med = np.median(prediction, axis=0)
std = np.std(prediction, axis=0)



# mean of values inside the standard mean
clipped_pres = np.clip(np.vstack(prediction), mean-std, mean+std)
clipped_mean = np.mean(clipped_pres, axis=0)

In [ ]:
med

In [ ]:
mean

# Submission

In [ ]:
submission = pd.DataFrame({'Id': sample_submission['Id'], 'Pawpularity': mean})
submission

In [ ]:
submission.to_csv("submission.csv", index=False)